In [1]:
import socket
from threading import Thread
import pickle
from util import *

BALANCE_SHEET = {}
CLIENT_MAP = {}

class Server_Thread(Thread):
    def __init__(self, connection, host, port):
        Thread.__init__(self)
        self.connection = connection
        self.host = name
        self.port = port

    def run(self):
        self.handle_messages()
        self.connection.close()

    def handle_messages(self):
        while True:
            request = self.connection.recv(buff_size)
            data = pickle.loads(request)
            if data.reqType == "BALANCE":
                self.get_balance(data)
            elif data.reqType == "ADD_TRANSACTION":
                self.add_transaction(data)

    def get_balance(self, data):
        CLIENT_MAP[data.fromPid].connection.sendall(str(BALANCE_SHEET[data.fromPid]))

    def add_transaction(self, data):
        BALANCE_SHEET[data.transaction.sender] -= data.transaction.amount
        BALANCE_SHEET[data.transaction.receiver] += data.transaction.amount
        CLIENT_MAP[data.fromPid].connection.sendall("SUCCESSFUL")

def printBalance():
    print("=========================================")
    for person in BALANCE_SHEET:
        print("{} : {}".format(CLIENT_INFO[person], BALANCE_SHEET[person]))
    print("=========================================")

def main():
    server_socket = socket.socket()
    try:
        server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        server_socket.bind((HOST, SERVER_PORT))
        print("Hosted the server at {}".format(SERVER_PORT))
    except socket.error as e:
        print(str(e))

    print("Waiting for the connection to clients")
    server_socket.listen(len(CLIENT_PORTS))

    BALANCE_SHEET = {x: 10 for x in range(1, len(CLIENT_PORTS)+1)}

    client_count = 0
    while client_count < len(CLIENT_PORTS):
        connection, client_address = server_socket.accept()
        if client_address[1] not in CLIENT_PORTS:
            print("Client - {} not in the set of allowed ports".format(client_address))
        print('Connected to: ' + client_address[0] + ':' + str(client_address[1]))
        client= Server_Thread(connection, client_address[0], client_address[1])
        client.start()
        CLIENT_MAP[client_address[1]%10] = client
        client_count+=1

    while True:
        print("type BALANCE to check the balance sheet")
        print("type Q to close the connection")
        user_input = input()
        if user_input == "BALANCE":
            print(printBalance())
        elif user_input == "Q":
            server_socket.close()
            for connection in CLIENT_MAP.values():
                connection.close()
            break


if __name__ == "__main__":
    main()

Hosted the server at 65500
Waiting for the connection to clients


KeyboardInterrupt: 